In [85]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re


In [86]:
df = pd.read_csv(r"C:\Users\ranga\OneDrive\Desktop\BMW-Tech-Works\Task 3\dataset\Parts.csv", delimiter=';')

In [87]:
# Basic dfset Info

print("dfset shape:", df.shape)
print("\nColumns and df Types:")
print(df.dtypes)
print("\nFirst 5 Rows of the dfset:")
df.head()

dfset shape: (998, 32)

Columns and df Types:
ID                                  object
DESCRIPTION                         object
Attribut1                           object
Additional Feature                  object
Application                         object
Characteristic                      object
Temp                                object
Height                              object
Length in mm                        object
Rating                              object
Material                            object
Size                                object
Code                                object
Joule-integral-Nom (J)              object
LC Risk                             object
Maximum AC Voltage Rating           object
Maximum DC Voltage Rating           object
Maximum Power Dissipation           object
Mounting                            object
Mounting Feature                    object
Number of Terminals                float64
Operating Temperature-Max (Cel)     object
Operatin

,ID,DESCRIPTION,Attribut1,Additional Feature,Application,Characteristic,Temp,Height,Length in mm,Rating,...,Operating Temperature-Min (Cel),Physical Dimension,Pre-arcing time-Min (ms),Product Diameter,Product Length,Rated Breaking Capacity (A),Rated Current (A),Rated Voltage (V),Rated Voltage(AC) (V),Rated Voltage(DC) (V)
0,A1,Indicator Red Fast Movement 1.6A 250V Holder P...,Fast,NaN,Primary Protection In Equipment,VERY FAST,NaN,20mm,5.2mm,1.6A,...,-55Cel,5.2mm x 20mm,3ms,5.2mm,20mm,1500A,1.6A,250V,250V,NaN
1,A2,"Non Resettable Indicators Electric Indicator, ...",NaN,NaN,Primary Protection In Equipment,VERY FAST,NaN,20mm,5.2mm,NaN,...,-55Cel,5.2mm x 20mm,3ms,5.2mm,20mm,1500A,6.3A,250V,250V,NaN
2,A3,Indicator Red Fast Movement 8A 250V Holder Pla...,Fast,NaN,Primary Protection In Equipment,VERY FAST,NaN,20mm,5.2mm,8A,...,-55Cel,5.2mm x 20mm,10ms,5.2mm,20mm,1500A,8A,250V,250V,NaN
3,A4,"Non Resettable Indicators Electric Indicator, ...",NaN,NaN,Primary Protection In Equipment,VERY FAST,NaN,20mm,5.2mm,NaN,...,-55Cel,5.2mm x 20mm,10ms,5.2mm,20mm,1500A,10A,250V,250V,NaN
4,A5,Indicator Red Fast Movement 12.5A 250V Holder ...,Fast,NaN,Primary Protection In Equipment,VERY FAST,NaN,20mm,5.2mm,12.5A,...,-55Cel,5.2mm x 20mm,10ms,5.2mm,20mm,500A,12.5A,250V,250V,NaN


In [88]:

missing_counts = df.isna().sum()
print("Missing values per column:")
print(missing_counts)
print("\nPercentage of missing values per column:")
print((missing_counts / len(df)) * 100)

Missing values per column:
ID                                   0
DESCRIPTION                        335
Attribut1                          228
Additional Feature                 674
Application                        177
Characteristic                     326
Temp                               581
Height                             293
Length in mm                       293
Rating                             164
Material                           227
Size                               118
Code                               443
Joule-integral-Nom (J)             335
LC Risk                            212
Maximum AC Voltage Rating          264
Maximum DC Voltage Rating          550
Maximum Power Dissipation          745
Mounting                           224
Mounting Feature                   319
Number of Terminals                240
Operating Temperature-Max (Cel)    291
Operating Temperature-Min (Cel)    296
Physical Dimension                 330
Pre-arcing time-Min (ms)           89

- For the analytics description is key column to find similar parts , 
- if we have NAN values there then it makes no sence to find similar parts, 
- lets not take mode of DESCRIPTION colun beacuse it adds Bias as the unique value counts in description is always 1


In [89]:
# Function to create a description from other columns
def create_description(row):
    desc = row.get("DESCRIPTION", "")
    # Check if DESCRIPTION is missing or empty (after stripping)
    if pd.isna(desc) or str(desc).strip() == "":
        # Get all columns except "ID" and "DESCRIPTION"
        cols_to_join = [col for col in row.index if col not in ['ID', 'DESCRIPTION']]
        # Collect non-empty, non-NaN values from these columns
        values = []
        for col in cols_to_join:
            val = row[col]
            if pd.notna(val) and str(val).strip() != "":
                values.append(str(val).strip())
        # Join the values with commas
        new_desc = ", ".join(values)
        # If no values were found, return an empty string (will be dropped later)
        return new_desc if new_desc != "" else np.nan
    else:
        return desc

# Create a new column with the updated description
df['new_DESCRIPTION'] = df.apply(create_description, axis=1)

# Drop rows where the new description is still missing (i.e. no data contributed)
before_drop = len(df)
df = df[df['new_DESCRIPTION'].notna() & (df['new_DESCRIPTION'].str.strip() != "")]
after_drop = len(df)
print(f"Dropped {before_drop - after_drop} rows that contributed no description information.")

# Update the DESCRIPTION column with the new data
df['DESCRIPTION'] = df['new_DESCRIPTION']
df.drop(columns=['new_DESCRIPTION'], inplace=True)


print(len(df))
df.head()



Dropped 6 rows that contributed no description information.
992


,ID,DESCRIPTION,Attribut1,Additional Feature,Application,Characteristic,Temp,Height,Length in mm,Rating,...,Operating Temperature-Min (Cel),Physical Dimension,Pre-arcing time-Min (ms),Product Diameter,Product Length,Rated Breaking Capacity (A),Rated Current (A),Rated Voltage (V),Rated Voltage(AC) (V),Rated Voltage(DC) (V)
0,A1,Indicator Red Fast Movement 1.6A 250V Holder P...,Fast,NaN,Primary Protection In Equipment,VERY FAST,NaN,20mm,5.2mm,1.6A,...,-55Cel,5.2mm x 20mm,3ms,5.2mm,20mm,1500A,1.6A,250V,250V,NaN
1,A2,"Non Resettable Indicators Electric Indicator, ...",NaN,NaN,Primary Protection In Equipment,VERY FAST,NaN,20mm,5.2mm,NaN,...,-55Cel,5.2mm x 20mm,3ms,5.2mm,20mm,1500A,6.3A,250V,250V,NaN
2,A3,Indicator Red Fast Movement 8A 250V Holder Pla...,Fast,NaN,Primary Protection In Equipment,VERY FAST,NaN,20mm,5.2mm,8A,...,-55Cel,5.2mm x 20mm,10ms,5.2mm,20mm,1500A,8A,250V,250V,NaN
3,A4,"Non Resettable Indicators Electric Indicator, ...",NaN,NaN,Primary Protection In Equipment,VERY FAST,NaN,20mm,5.2mm,NaN,...,-55Cel,5.2mm x 20mm,10ms,5.2mm,20mm,1500A,10A,250V,250V,NaN
4,A5,Indicator Red Fast Movement 12.5A 250V Holder ...,Fast,NaN,Primary Protection In Equipment,VERY FAST,NaN,20mm,5.2mm,12.5A,...,-55Cel,5.2mm x 20mm,10ms,5.2mm,20mm,500A,12.5A,250V,250V,NaN


In [92]:
#  [x for x in df['Attribut1'].unique() if x != np.nan]

#  data = ['Fast', float('nan'), 'Slow Blow', 'Very Fast', 'Medium Time Delay', 'Super Time Lag']

non_nan_elements = [item for item in df['Characteristic'].unique() if not isinstance(item, float) or not item != item]

print(non_nan_elements)

['VERY FAST', 'TIME LAG', 'SUPER FAST', 'FAST', 'MEDIUM TIME LAG', 'SLOW', 'MEDIUM']


In [93]:

# List of patterns to search for
attribute_patterns = [item for item in df['Attribut1'].unique() if not isinstance(item, float) or not item != item]

# Function to extract attribute type based on pattern matching
def extract_attribute(description):
    if pd.isna(description):
        return "NA"
    
    # Get text before first comma
    text_before_comma = description.split(',')[0] if ',' in description else description
    
    # Check for each pattern in the text
    for pattern in attribute_patterns:
        if re.search(r'\b' + re.escape(pattern) + r'\b', text_before_comma, re.IGNORECASE):
            return pattern
    
    # If no match found, return NA
    return "NA"

# Create a backup of the original column
df['Attribut1_Original'] = df['Attribut1']

# Only fill values that are empty or NaN
mask = df['Attribut1'].isna() | (df['Attribut1'] == '')
df.loc[mask, 'Attribut1'] = df.loc[mask, 'DESCRIPTION'].apply(extract_attribute)

# Count statistics
total_rows = len(df)
originally_filled = total_rows - sum(df['Attribut1_Original'].isna() | (df['Attribut1_Original'] == ''))
newly_filled_na = sum((df['Attribut1'] == 'NA') & (df['Attribut1_Original'].isna() | (df['Attribut1_Original'] == '')))

# Calculate counts for each attribute pattern
pattern_counts = {}
for pattern in attribute_patterns:
    pattern_counts[pattern] = sum((df['Attribut1'] == pattern) & 
                                 (df['Attribut1_Original'].isna() | (df['Attribut1_Original'] == '')))

total_newly_filled = sum(pattern_counts.values()) + newly_filled_na
total_filled_now = sum(~df['Attribut1'].isna() & (df['Attribut1'] != ''))
still_empty = total_rows - total_filled_now

# Print statistics
print(f"Total rows in dataset: {total_rows}")
print(f"Originally filled Attribut1 values: {originally_filled}")
print("\nNewly filled attribute counts:")
for pattern, count in pattern_counts.items():
    print(f"  - '{pattern}': {count}")
print(f"  - 'NA' (non-matching): {newly_filled_na}")
print(f"\nTotal newly filled values: {total_newly_filled}")
print(f"Total filled values after processing: {total_filled_now}")
print(f"Still empty values: {still_empty}")
print(f"Percentage of dataset now filled: {total_filled_now/total_rows*100:.2f}%")

Total rows in dataset: 992
Originally filled Attribut1 values: 770

Newly filled attribute counts:
  - 'Fast': 12
  - 'Slow Blow': 1
  - 'Very Fast': 0
  - 'Medium Time Delay': 0
  - 'Super Time Lag': 0
  - 'NA' (non-matching): 209

Total newly filled values: 222
Total filled values after processing: 992
Still empty values: 0
Percentage of dataset now filled: 100.00%


In [94]:
import pandas as pd
import re

def extract_additional_features(description):
    """
    Extract rated breaking capacity from description using pattern matching
    Returns formatted string if matches found, otherwise None
    """
    if pd.isna(description):
        return None
        
    # Split description into components
    parts = [p.strip() for p in description.split(',')]
    
    # Regular expression patterns
    voltage_pattern = r'(\d+\.?\d*)\s*(VAC|VDC)'
    current_pattern = r'(\d+\.?\d*)A\s*\(IR\)'
    
    # Storage for matches
    voltages = []
    breaking_current = None
    
    # Scan through all description parts
    for part in parts:
        # Check for voltage matches
        voltage_match = re.search(voltage_pattern, part, re.IGNORECASE)
        if voltage_match:
            value, v_type = voltage_match.groups()
            voltages.append((v_type.upper(), f"{value} {v_type.upper()}"))
        
        # Check for breaking current
        current_match = re.search(current_pattern, part, re.IGNORECASE)
        if current_match:
            breaking_current = f"{current_match.group(1)} A"
    
    # Prioritize VDC over VAC if both exist
    selected_voltage = None
    for v_type, voltage in reversed(voltages):  # Check last occurrence first
        if v_type == 'VDC':
            selected_voltage = voltage
            break
    if not selected_voltage:
        for v_type, voltage in reversed(voltages):
            if v_type == 'VAC':
                selected_voltage = voltage
                break
    
    # Create formatted string if both components found
    if selected_voltage and breaking_current:
        return f"RATED BREAKING CAPACITY AT {selected_voltage}: {breaking_current}"
    
    return "NA"  # Return NA instead of None for failed matches


# Apply the extraction function
original_empty = df['Additional Feature'].isna().sum()
df['New_Additional_Feature'] = df['DESCRIPTION'].apply(extract_additional_features)

# Update only empty original Additional Feature entries
update_mask = df['Additional Feature'].isna()
df.loc[update_mask, 'Additional Feature'] = df.loc[update_mask, 'New_Additional_Feature']
df = df.drop(columns=['New_Additional_Feature'])

# Handle remaining NA values
df['Additional Feature'].fillna("NA", inplace=True)

# Calculate statistics
filled_count = update_mask.sum()
successfully_filled = (df['Additional Feature'] != "NA").sum() - (original_empty - update_mask.sum())
remaining_empty = (df['Additional Feature'] == "NA").sum()

print(f"Filling Statistics:")
print(f"Total empty entries before: {original_empty}")
print(f"Successfully filled entries: {successfully_filled}")
print(f"NA entries: {remaining_empty}")
print(f"Success rate: {(successfully_filled/original_empty*100 if original_empty else 0):.2f}%")

# Show samples
print("\nSample filled entry:")
filled_sample = df[update_mask].iloc[0:1]
print(filled_sample[['DESCRIPTION', 'Additional Feature']].to_string(index=False))


Filling Statistics:
Total empty entries before: 668
Successfully filled entries: 610
NA entries: 382
Success rate: 91.32%

Sample filled entry:
                                                                                                                                                                    DESCRIPTION                         Additional Feature
Indicator Red Fast Movement 1.6A 250V Holder Plastic 5 X 20mm Ceramic Box CCC/PSE/VDE/cULus Electric Indicator, Very Fast Blow, 1.6A, 250VAC, 1500A (IR), Inline/holder, 5x20mm RATED BREAKING CAPACITY AT 250 VAC: 1500 A


C:\Users\ranga\AppData\Local\Temp\ipykernel_26632\1918610229.py:65: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Additional Feature'].fillna("NA", inplace=True)


In [ ]:
def fill_na_columns(df, columns):
 
    for col in columns:
        if col in df.columns:
            df[col] = df[col].fillna("NA")
    return df

# List of columns to fill with "NA" wherever there is a NaN value.
cols_to_fill = ["Application", "Material", "Code", "Joule-integral-Nom (J)","LC Risk","Mounting","Mounting Feature",
"Operating Temperature-Max (Cel)","Operating Temperature-Min (Cel)","Physical Dimension"]

# Fill missing values for the specified columns.
df = fill_na_columns(df, cols_to_fill)


In [ ]:
["Number of Terminals"] # take mean

In [71]:
# List of patterns to search for in Characteristic
characteristic_patterns = [item for item in df['Characteristic'].unique() 
                          if not isinstance(item, float) or not item != item]

# Modified function to check parts AFTER first comma
def extract_characteristic(description):
    if pd.isna(description):
        return "NA"
    
    # Split into parts and exclude first chunk
    parts = description.split(',')
    text_after_first_comma = ','.join(parts[1:]) if len(parts) > 1 else ''
    
    # Check each pattern in remaining text
    for pattern in characteristic_patterns:
        if re.search(r'\b' + re.escape(str(pattern)) + r'\b', text_after_first_comma, re.IGNORECASE):
            return pattern
    
    return "NA"

# Create backup column
df['Characteristic_Original'] = df['Characteristic']

# Apply to empty values
mask = df['Characteristic'].isna() | (df['Characteristic'] == '')
df.loc[mask, 'Characteristic'] = df.loc[mask, 'DESCRIPTION'].apply(extract_characteristic)

# Count statistics
total_rows = len(df)
originally_filled = total_rows - sum(df['Characteristic_Original'].isna() | (df['Characteristic_Original'] == ''))
newly_filled_na = sum((df['Characteristic'] == 'NA') & (df['Characteristic_Original'].isna() | (df['Characteristic_Original'] == '')))

# Calculate pattern counts
char_pattern_counts = {}
for pattern in characteristic_patterns:
    char_pattern_counts[pattern] = sum((df['Characteristic'] == pattern) & 
                                   (df['Characteristic_Original'].isna() | (df['Characteristic_Original'] == '')))

total_newly_filled = sum(char_pattern_counts.values()) + newly_filled_na
total_filled_now = sum(~df['Characteristic'].isna() & (df['Characteristic'] != ''))
still_empty = total_rows - total_filled_now

# Print statistics
print(f"Total rows in dataset: {total_rows}")
print(f"Originally filled Characteristic values: {originally_filled}")
print("\nNewly filled characteristic counts:")
for pattern, count in char_pattern_counts.items():
    print(f"  - '{pattern}': {count}")
print(f"  - 'NA' (non-matching): {newly_filled_na}")
print(f"\nTotal newly filled values: {total_newly_filled}")
print(f"Total filled values after processing: {total_filled_now}")
print(f"Still empty values: {still_empty}")
print(f"Percentage of dataset now filled: {total_filled_now/total_rows*100:.2f}%")

Total rows in dataset: 998
Originally filled Characteristic values: 672

Newly filled characteristic counts:
  - 'VERY FAST': 0
  - 'TIME LAG': 0
  - 'SUPER FAST': 0
  - 'FAST': 0
  - 'MEDIUM TIME LAG': 0
  - 'SLOW': 0
  - 'MEDIUM': 0
  - 'NA' (non-matching): 326

Total newly filled values: 326
Total filled values after processing: 998
Still empty values: 0
Percentage of dataset now filled: 100.00%


In [76]:
import pandas as pd
import re

# Define two patterns:
# Pattern for three numbers (Temp x Height x Length in mm)
pattern_three = re.compile(r'(\d+(?:\.\d+)?)\s*[xX×]\s*(\d+(?:\.\d+)?)\s*[xX×]\s*(\d+(?:\.\d+)?)\s*mm')
# Pattern for two numbers (Height x Length in mm)
pattern_two = re.compile(r'(\d+(?:\.\d+)?)\s*[xX×]\s*(\d+(?:\.\d+)?)\s*mm')

def extract_dimensions_from_description(desc):
    """
    Given a description string, try to extract dimensions.
    If a three-number pattern is found, return (Temp, Height, Length, "three").
    If only a two-number pattern is found, assume they represent Height and Length,
    and return ("NA", Height, Length, "two").
    If no pattern is found, return ("NA", "NA", "NA", "none").
    """
    if not isinstance(desc, str):
        return ("NA", "NA", "NA", "none")
    
    m_three = pattern_three.search(desc)
    if m_three:
        return m_three.group(1), m_three.group(2), m_three.group(3), "three"
    
    m_two = pattern_two.search(desc)
    if m_two:
        return "NA", m_two.group(1), m_two.group(2), "two"
    
    return ("NA", "NA", "NA", "none")



# Ensure the DESCRIPTION column exists and fill missing values with an empty string.
if 'DESCRIPTION' not in df.columns:
    raise KeyError("The CSV file must contain a 'DESCRIPTION' column.")
df['DESCRIPTION'] = df['DESCRIPTION'].fillna("")

# -----------------------------
# Extract Dimensions and Collection Extraction Type
# -----------------------------
# Apply the extraction function and expand the result into new columns.
extraction_results = df['DESCRIPTION'].apply(extract_dimensions_from_description)
df[['Temp_extracted', 'Height_extracted', 'Length_in_mm_extracted', 'Extraction_type']] = pd.DataFrame(extraction_results.tolist(), index=df.index)

# -----------------------------
# Print Extraction Statistics
# -----------------------------
total_rows = len(df)
count_three = (df['Extraction_type'] == 'three').sum()
count_two   = (df['Extraction_type'] == 'two').sum()
count_none  = (df['Extraction_type'] == 'none').sum()

print("Total rows:", total_rows)
print("Rows filled by three-number pattern:", count_three, f"({(count_three/total_rows)*100:.1f}%)")
print("Rows filled by two-number pattern:", count_two, f"({(count_two/total_rows)*100:.1f}%)")


# -----------------------------
# Update or Create the Target Columns
# -----------------------------
# For each target column (Temp, Height, Length in mm), fill missing values with extracted data.
# If the column already exists, fill NaNs; if not, create the column.
for col, extracted_col in zip(["Temp", "Height", "Length in mm"],
                              ["Temp_extracted", "Height_extracted", "Length_in_mm_extracted"]):
    if col in df.columns:
        df[col] = df[col].fillna(df[extracted_col])
    else:
        df[col] = df[extracted_col]

# Optionally, drop the temporary extracted columns if no longer needed.
df.drop(columns=["Temp_extracted", "Height_extracted", "Length_in_mm_extracted", "Extraction_type"], inplace=True)

# Display a sample of the updated DataFrame.
print("\nSample of updated data with dimensions:")
print(df[["DESCRIPTION", "Temp", "Height", "Length in mm"]].head())


Total rows: 998
Rows filled by three-number pattern: 83 (8.3%)
Rows filled by two-number pattern: 203 (20.3%)

Sample of updated data with dimensions:
                                         DESCRIPTION Temp Height Length in mm
0  Indicator Red Fast Movement 1.6A 250V Holder P...   NA   20mm        5.2mm
1  Non Resettable Indicators Electric Indicator, ...   NA   20mm        5.2mm
2  Indicator Red Fast Movement 8A 250V Holder Pla...   NA   20mm        5.2mm
3  Non Resettable Indicators Electric Indicator, ...   NA   20mm        5.2mm
4  Indicator Red Fast Movement 12.5A 250V Holder ...   NA   20mm        5.2mm


In [75]:
df.to_csv("Parts_processed.csv", index=False)

In [38]:
import pandas as pd
import re
import numpy as np


def extract_dimensions(description):
    """
    Extract dimensions (length x width x height in mm) from the description.
    Example match: "9.73 X 5.03 X 3.81mm"
    Returns a tuple of three floats or (np.nan, np.nan, np.nan) if not found.
    """
    pattern = re.compile(r'(\d+\.?\d*)\s*[xX×]\s*(\d+\.?\d*)\s*[xX×]\s*(\d+\.?\d*)\s*mm')
    match = pattern.search(description)
    if match:
        return tuple(float(x) for x in match.groups())
    return (np.nan, np.nan, np.nan)

def extract_current(description):
    """
    Extract the current rating from the description.
    Looks for a pattern like "5A" and returns the float value.
    """
    pattern = re.compile(r'(\d+\.?\d*)\s*A\b')
    match = pattern.search(description)
    if match:
        return float(match.group(1))
    return np.nan

def extract_numeric_from_rating(rating_str):
    """
    Given a string rating like '1.6A', extract the numeric part and convert to float.
    """
    if pd.isna(rating_str):
        return np.nan
    match = re.search(r'(\d+\.?\d*)', str(rating_str))
    if match:
        return float(match.group(1))
    return np.nan

def extract_mounting(description):
    """
    Extract mounting type from description by checking for key terms.
    """
    mounting_types = ['smd', 'surface mount', 'through hole', 'radial dip']
    desc_lower = description.lower()
    for m in mounting_types:
        if m in desc_lower:
            return m.title()  # Standardize capitalization (e.g., "Smd" -> "Smd" or "Surface Mount")
    return np.nan

def extract_movement(description):
    """
    Extract movement type from description (e.g., "Slow Blow" or "Fast Blow").
    """
    desc_lower = description.lower()
    if 'slow blow' in desc_lower:
        return 'Slow Blow'
    elif 'fast blow' in desc_lower:
        return 'Fast Blow'
    return np.nan


In [39]:
if 'DESCRIPTION' not in df.columns:
    raise KeyError("The CSV file must contain a 'DESCRIPTION' column.")
df['DESCRIPTION'] = df['DESCRIPTION'].fillna("")

# Optionally, create a cleaned version of DESCRIPTION for text parsing (e.g., lowercase)
df['description_clean'] = df['DESCRIPTION'].str.lower()

# -----------------------------
# Extracting and Imputing Missing Values
# -----------------------------

# 1. Extract Dimensions: Create new columns for Length, Width, and Height in mm.
dimensions = df['DESCRIPTION'].apply(extract_dimensions)
df[['Length_mm', 'Width_mm', 'Height_mm']] = pd.DataFrame(dimensions.tolist(), index=df.index)

# 2. Process the Rating column:
#    If Rating is missing or has non-numeric content (like "1.6A"), extract numeric value from DESCRIPTION.
if 'Rating' not in df.columns:
    df['Rating'] = np.nan

df['Rating'] = df.apply(
    lambda row: extract_current(row['DESCRIPTION']) if pd.isna(row['Rating']) 
                else extract_numeric_from_rating(row['Rating']),
    axis=1
)

# 3. Extract Mounting Type:
if 'Mounting' not in df.columns:
    df['Mounting'] = np.nan

df['Mounting'] = df.apply(
    lambda row: extract_mounting(row['DESCRIPTION']) if pd.isna(row['Mounting']) else row['Mounting'],
    axis=1
)

# 4. Extract Movement Type:
# Create a new column 'Movement' if it doesn't exist
if 'Movement' not in df.columns:
    df['Movement'] = np.nan

df['Movement'] = df.apply(
    lambda row: extract_movement(row['DESCRIPTION']) if pd.isna(row['Movement']) else row['Movement'],
    axis=1
)

# 5. For other categorical columns (example: Material, Application, Characteristic)
categorical_columns = ['Material', 'Application', 'Characteristic']
for col in categorical_columns:
    if col in df.columns:
        df[col] = df[col].fillna("Unknown")

# 6. For additional numerical columns (example: 'Operating Temperature-Max (Cel)', etc.)
numerical_columns = ['Operating Temperature-Max (Cel)', 'Operating Temperature-Min (Cel)', 'Joule-integral-Nom (J)']
for col in numerical_columns:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')
        median_val = df[col].median()
        df[col] = df[col].fillna(median_val)

# -----------------------------
# Save and Display Processed df
# -----------------------------

# Save the processed DataFrame to a new CSV file
df.to_csv("Parts_processed.csv", index=False)

df.head()

c:\Users\ranga\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:1231: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
c:\Users\ranga\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:1231: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
c:\Users\ranga\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:1231: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


PermissionError: [Errno 13] Permission denied: 'Parts_processed.csv'